import packages

In [1]:
import os
os.chdir('..')

from analysis.fidelity_prediction import FidelityModel
from baselines.fidelity_prediction.rb_prediction import RBModel
from simulator.gate_error_model import GateErrorModel

from analysis.vectorization import RandomwalkModel

from data_objects.random_circuit import random_circuits, random_circuit
from data_objects.backend import Backend, LinearBackend, GridBackend, FullyConnectedBackend

from simulator.noisy_simulator import NoisySimulator
import random
from dataset import load_dataset
from tools.ray_func import map


model settings

In [2]:
n_qubits = 18
n_steps = 2
n_walks = 20
backend = GridBackend(3, 6)


load dataset with ground truth fidelity

In [3]:
from data_objects.circuit import SeperatableCircuit

dataset_id = '20230321'
circuits: list[SeperatableCircuit] = load_dataset(dataset_id)

train upstream model, turn a circuit to vectors using random walk

In [4]:
vec_model = RandomwalkModel(n_steps = n_steps, n_walks = n_walks, backend = backend)
gate_vecs_per_circuit = vec_model.train(circuits, True, remove_redundancy = False)

start random walk for 2500 circuits


/home/langcongliang/enter/envs/janusq/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-03-29 18:00:50,806	INFO worker.py:1724 -- Started a local Ray instance.
100%|██████████| 10/10 [01:50<00:00, 11.06s/it]


count path
device size after random walk = 41
0's path table size = 16761
1's path table size = 16765
2's path table size = 17614
3's path table size = 17207
4's path table size = 19494
5's path table size = 19421
6's path table size = 16949
7's path table size = 17406
8's path table size = 16983
9's path table size = 21902
10's path table size = 19675
11's path table size = 19693
12's path table size = 17991
13's path table size = 20824
14's path table size = 20799
15's path table size = 23123
16's path table size = 21958
17's path table size = 19434
(6, 12)'s path table size = 5264
(3, 4)'s path table size = 3290
(16, 17)'s path table size = 3080
(12, 13)'s path table size = 4097
(2, 8)'s path table size = 3899
(11, 17)'s path table size = 6336
(13, 14)'s path table size = 6874
(7, 13)'s path table size = 4035
(4, 5)'s path table size = 4008
(3, 9)'s path table size = 3321
(14, 15)'s path table size = 4065
(0, 1)'s path table size = 3260
(1, 2)'s path table size = 6307
(10, 11)'s pat

select interaction patterns randomly, simulate interaction between gates

In [5]:
flaten_circuits, flaten_fidelities = [], []
for circuit in circuits:
    for sub_cir in circuit.seperatable_circuits:
        flaten_circuits.append(sub_cir)
        flaten_fidelities.append(sub_cir.ground_truth_fidelity)

train fidelity prediction model

In [6]:
fidelity_model = FidelityModel(vec_model)
fidelity_model.train((flaten_circuits, flaten_fidelities))

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


len(train dataset) = 8000, len(validation dataset) = 2000


8000it [00:00, 1084009.56it/s]
100%|██████████| 20/20 [00:00<00:00, 139.36it/s]


(_map pid=3435886) /home/langcongliang/janusq/analysis/fidelity_prediction.py:84: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in array is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
(_map pid=3435886)   return jnp.array(vec_model.vectorize(circuit), jnp.float64)
(_map pid=3435886) An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
(_map pid=3435737) /home/langcongliang/janusq/analysis/fidelity_prediction.py:84: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in array is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current

n_gates_list: [20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150]
n_gates_count_list: [199, 770, 760, 764, 765, 769, 772, 734, 652, 381, 386, 381, 374, 293]


2000it [00:00, 238672.09it/s]
100%|██████████| 5/5 [00:00<00:00, 45.36it/s]


n_gates_list: [20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150]
n_gates_count_list: [41, 190, 200, 196, 195, 191, 188, 206, 128, 99, 94, 99, 106, 67]
epoch: 0, 	 epoch loss = 48583.08984375, 	 validation loss = 3.815769910812378


predict on test dataset

compare with RB predict model

In [22]:
from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import numpy as np
import random
def plot_scaater(reals, predicts, durations ,name):
    par = np.polyfit(reals, predicts, 1, full=True)
    slope=par[0][0]
    intercept=par[0][1]
    x1 = [0.4, 1.0]
    y1 = [slope*xx + intercept  for xx in x1]
    #定义颜色
    colors = ["#FF3636", '#277C8E' ,"#1F77B4"]
    '''xia <- shang'''
    # colors.reverse()
    # colors = np.array(colors) / 256
    # 定义颜色的位置
    pos = [0, .5, 1]
    # 创建colormap对象
    cmap = LinearSegmentedColormap.from_list('my_colormap', list(zip(pos, colors)))

    normalied_durations = (durations - durations.min())/(durations.max() - durations.min())

    # cmap_name = 'Blues'
    # cmap_name = 'viridis'
    # cmap_name = 'plasma'
    # cmap_name = 'winter'

    random_index = list(range(len(reals)))
    random.shuffle(random_index)
    random_index = random_index[:1500]
    reals = np.array(reals)
    predicts = np.array(predicts)
    fig, axes = plt.subplots(figsize=(10, 10))  # 创建一个图形对象和一个子图对象
    axes.axis([0, 1, 0, 1])
    axes.scatter(reals[random_index], predicts[random_index], c= normalied_durations[random_index], cmap=cmap,alpha = 0.6, s=80 )
    axes.plot(x1,y1)
    axes.set_xlim(.2, 1)
    axes.set_ylim(.2, 1)
    axes.set_xlabel('real ')
    axes.set_ylabel('predict')
    axes.plot([[0,0],[1,1]])
    # fig.colorbar(cm.ScalarMappable( cmap=cmap))
    fig.savefig(name)
    print(slope, intercept)